In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import os
try:
    os.chdir('../data/preprocess0506')
except:
    pass

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
prices = pd.read_csv('./final_processed/daily_prices.csv', parse_dates=['date']).sort_values(['ticker','date']).set_index('date')
# firms = pd.read_csv('./final_processed/firm_ratios.csv',  parse_dates=['date']).sort_values(['ticker','date']).set_index('date')
fr = pd.read_csv('../zacks/fr.csv')

In [3]:
fr = pd.read_csv('../zacks/fr.csv')

In [4]:
fr

,"No Data, try with a different combination of filters"


In [4]:
period = ['2000-01','2000-03']

In [43]:
df = prices.loc[period[0]:period[1]].copy()
df2 = firms.loc[period[0]:period[1]].copy()

/var/folders/1t/_7p_zm4x449blqs7bvqvb0rm0000gn/T/ipykernel_42080/714610207.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df = prices.loc[period[0]:period[1]].copy()
/var/folders/1t/_7p_zm4x449blqs7bvqvb0rm0000gn/T/ipykernel_42080/714610207.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df2 = firms.loc[period[0]:period[1]].copy()


In [38]:
def get_log_return(df, col = 'adj_close'):
    df = df.pivot_table(index = 'date', columns = 'ticker', values = col)
    ret = (np.log(df) - np.log(df.shift(1))).cumsum().dropna()
    ret = ret.unstack().to_frame('log_return_'+col).reset_index(0)
    return ret

data = get_log_return(df, col = 'adj_close')

In [46]:
data = pd.merge(data, df2, on=['ticker','date'], how='left').fillna(0)

In [66]:
df2 = df2.fillna(0)
fit_data = df2.drop(['ticker','gicdesc'], axis=1)

In [51]:
pca = PCA(n_components=2)

In [57]:
df2_trans = pca.fit_transform(df2.drop(['ticker','gicdesc'], axis=1))

In [68]:
scale = StandardScaler()

In [71]:
scale.fit_transform(fit_data)

array([[-0.31146464, -0.49183917, -0.93090279, ..., -0.52665296,
        -0.35172902, -0.80914574],
       [-0.31146464, -0.49183917, -0.93090279, ..., -0.52665296,
        -0.35172902, -0.80914574],
       [-0.31146464, -0.49183917, -0.93090279, ..., -0.52665296,
        -0.35172902, -0.80914574],
       ...,
       [-0.12170012, -0.49183917, -0.93090279, ..., -0.52665296,
        -0.35172902, -0.80914574],
       [-0.12170012, -0.49183917, -0.93090279, ..., -0.52665296,
        -0.35172902, -0.80914574],
       [-0.09861124, -0.49183917, -0.93090279, ..., -0.52665296,
        -0.35172902, -0.80914574]])

In [60]:
pd.DataFrame(df2_trans, columns=['pca1','pca2'], index=df2.index)

,pca1,pca2
date,,
2000-01-03,-21649.579940,-2115.153576
2000-01-04,-21649.579940,-2115.153577
2000-01-05,-21649.579940,-2115.153577
2000-01-06,-21649.579940,-2115.153577
2000-01-07,-21649.579940,-2115.153577
...,...,...
2000-03-27,-17565.857265,-2530.841520
2000-03-28,-17565.857265,-2530.841520
2000-03-29,-17565.857265,-2530.841520
